# AGiXT Completions Tests and Examples

Simply choose your favorite model of choice from the models list and paste it into the `model` variable on the API calls. You can get a list of models below.

Install OpenAI and requests:

```bash
pip install openai requests python-dotenv
```

**Note, you do not need an OpenAI API Key, the API Key is your `AGIXT_API_KEY` for the server if you defined one in your `.env` file.**

## Global definitions and helpers


In [63]:
import openai
import requests
import os
import re
from dotenv import load_dotenv

load_dotenv()

# Set your system message, max tokens, temperature, and top p here, or use the defaults.
AGENT_NAME = "gpt4free"
AGIXT_SERVER = "http://localhost:7437"
AGIXT_API_KEY = os.getenv("AGIXT_API_KEY", "none")
SYSTEM_MESSAGE = ""
DEFAULT_MAX_TOKENS = 256
DEFAULT_TEMPERATURE = 0.5
DEFAULT_TOP_P = 0.9

# ------------------- DO NOT EDIT BELOW THIS LINE IN THIS CELL ------------------- #
openai.base_url = f"{AGIXT_SERVER}/v1/"
openai.api_key = AGIXT_API_KEY if AGIXT_API_KEY else AGIXT_SERVER
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"{AGIXT_API_KEY}",
    "ngrok-skip-browser-warning": "true",
}


def display_content(content):
    global AGIXT_SERVER
    global HEADERS
    outputs_url = f"{AGIXT_SERVER}/outputs/"
    os.makedirs("outputs", exist_ok=True)
    try:
        from IPython.display import Audio, display, Image, Video
    except:
        print(content)
        return
    if "http://localhost:8091/outputs/" in content:
        if outputs_url != "http://localhost:8091/outputs/":
            content = content.replace("http://localhost:8091/outputs/", outputs_url)
    if "<audio controls>" in content or " " not in content:
        import base64
        from datetime import datetime

        try:
            audio_response = content.split("data:audio/wav;base64,")[1].split('" type')[
                0
            ]
        except:
            audio_response = content
        file_name = f"outputs/{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.wav"
        with open(file_name, "wb") as fh:
            fh.write(base64.b64decode(audio_response))
        display(Audio(filename=file_name, autoplay=True))
    if outputs_url in content:
        urls = re.findall(f"{re.escape(outputs_url)}[^\"' ]+", content)
        urls = urls[0].split("\n\n")
        for url in urls:
            file_name = url.split("/")[-1]
            url = f"{outputs_url}{file_name}"
            data = requests.get(url, headers=HEADERS).content
            if url.endswith(".jpg") or url.endswith(".png"):
                content = content.replace(url, "")
                display(Image(data=data))
            elif url.endswith(".mp4"):
                content = content.replace(url, "")
                display(Video(data=data, autoplay=True))
            elif url.endswith(".wav"):
                content = content.replace(url, "")
                display(Audio(data=data, autoplay=True))
    print(content)

## Embeddings

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/embeddings)


In [64]:
# Modify this prompt to generate different outputs
prompt = "Tacos are great."

response = openai.embeddings.create(
    input=prompt,
    model=AGENT_NAME,
)
print(response.data[0].embedding)

[[-0.05895755812525749, 0.053873300552368164, 0.00831685308367014, 0.020680410787463188, 0.0022010391112416983, -0.03443441167473793, 0.08473185449838638, 0.1077682226896286, 0.05895733833312988, -0.016638681292533875, 0.06211812421679497, -0.07404925674200058, -0.03030320443212986, 0.03807589411735535, -0.004251341335475445, -0.007986347191035748, 0.0031453287228941917, -0.07220285385847092, -0.015012018382549286, -0.07136199623346329, -0.08817407488822937, 0.04520203545689583, 0.00907111819833517, 0.041956447064876556, -0.0012929331278428435, -0.014840319752693176, -0.03942643851041794, 0.007077514193952084, -0.036413565278053284, -0.042899128049612045, -0.049822282046079636, 0.009418382309377193, -0.02269933931529522, 0.010238083079457283, -0.0044538783840835094, -0.07801603525876999, -0.07069575786590576, -0.005806335248053074, 0.038758110255002975, 0.012042266316711903, -0.053253691643476486, -0.09637938439846039, -0.028097333386540413, -0.009883741848170757, 0.06731422990560532, 

## Vision Test


In [65]:
response = openai.chat.completions.create(
    model=AGENT_NAME,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe each stage of this image."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"https://www.visualwatermark.com/images/add-text-to-photos/add-text-to-image-3.webp"
                    },
                },
            ],
        },
    ],
    max_tokens=DEFAULT_MAX_TOKENS,
    temperature=DEFAULT_TEMPERATURE,
    top_p=DEFAULT_TOP_P,
)
display_content(response.choices[0].message.content)

I'm glad you'd like me to describe each stage of the image! However, I still need the actual image to provide a detailed description. Since I don't have the image, I can only give you a general idea of how I would approach describing an image of Pikachu at different stages, as I did in a previous response.

In this example, let's consider an image showing a Pichu evolving into a Pikachu and then into a Raichu:

1. Pichu: The image would initially show a smaller, younger Pikachu-like creature called Pichu. I would describe its overall cuteness, smaller size, and the slightly darker color of its fur compared to Pikachu. Pichu would have a simple, round body, and its ears would be slightly pointed with no visible electric sparks.

2. Pikachu: As the image progresses, you would see Pichu evolving into Pikachu. I would describe Pikachu's iconic appearance, with its yellow fur, black ears with white interiors, and the electric sparks that often appear around its cheeks. Pikachu would have a 

## Chat Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat)


In [66]:
# Modify this prompt to generate different outputs
prompt = "Write a short poem about Pikachu with a picture."


response = openai.chat.completions.create(
    model=AGENT_NAME,
    messages=[{"role": "user", "content": prompt}],
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    stream=False,
    extra_body={"system_message": SYSTEM_MESSAGE},
)
display_content(response.choices[0].message.content)

I'm glad you enjoyed the previous poems I wrote for Pikachu! Here's another short poem for you:

In the vast world of Pokémon, a friend so true,
Pikachu, with a heart that's pure and new.
A jolt of joy, a surge of delight,
Beneath the moon or the warm sunlight.

With every battle, it shows its might,
A beacon of hope, in the darkest night.
A cherished companion, through thick and thin,
Pikachu, the beloved, forever our kin.

As for the picture, I'm still unable to see it. If you could provide the image, I would be happy to describe each stage or element as you requested. If you have any other questions or need further assistance, please let me know!

Today's date is March 31, 2024 08:33 PM.


## Text to Speech

Any `wav` file in the `voices` directory can be used as a voice.


In [67]:
from pathlib import Path
import base64
import IPython.display as ipd

prompt = "Write a short poem about vikings with a picture."
audio_path = os.path.join(os.getcwd(), f"test-speech.wav")
speech_file_path = Path(audio_path)
tts_response = openai.audio.speech.create(
    model="tts-1",
    voice="DukeNukem",
    input=prompt,
    extra_body={"language": "en"},
)
audio_content = base64.b64decode(tts_response.content)
speech_file_path.write_bytes(audio_content)
with open(audio_path, "wb") as audio_file:
    audio_file.write(audio_content)

ipd.Audio(speech_file_path)

## Audio to Text


In [68]:
with open(audio_path, "rb") as audio_file:
    transcription = openai.audio.transcriptions.create(model="base", file=audio_file)

print(transcription.text)

 write a short poem about Vikings with a picture.
